# DIPC ROLLING AVERAGE SOLVER   
So you want to solve the WESM rolling average, but you are not enrolled in the IEMOP data service?    
Fear not, the data is readily available to YOU.    
All you have to do is get the DIPC files from the last 4 days from the [IEMOP site](https://www.iemop.ph/market-data/dipc-energy-results-raw/) and run this code! Ideally you should download from hour 1 and then just add the unposted time intervals for the current date manually.        
    
    
TLDR: This script solves 3 day rolling average so you can predict when the secondary price cap will trigger. 

In [1]:
import pandas as pd
from glob import glob
pd.set_option("display.max_columns", None) #show all columns

In [2]:
# create list of all DIPC csv files
DIPC_files = sorted(glob('DIPCER_*'))
DIPC_files[0:4]

['DIPCER_202301150100.csv',
 'DIPCER_202301150200.csv',
 'DIPCER_202301150300.csv',
 'DIPCER_202301150400.csv']

In [3]:
# load and concatinate all DIPC csv files into one dataframe
DIPC = pd.concat((pd.read_csv(file, skipfooter=1).assign(filename = file)
          for file in DIPC_files),ignore_index = True)
print(DIPC.columns)
print(DIPC.shape)
DIPC.head()

<ipython-input-3-b4cb4a89f192>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  DIPC = pd.concat((pd.read_csv(file, skipfooter=1).assign(filename = file)


Index(['TIME_INTERVAL', 'REGION_NAME', 'RESOURCE_NAME', 'PRICING_FLAG', 'LMP',
       'SCHED_MW', 'LMP_SMP', 'LMP_LOSS', 'LMP_CONGESTION', 'Unnamed: 9',
       'filename'],
      dtype='object')
(1057536, 11)


,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,PRICING_FLAG,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,filename
0,1/15/2023 12:05:00 AM,LUZON,01ACNPC_G01,OK,6585.7962,1.5000,6647.619,-61.8229,0.0,NaN,DIPCER_202301150100.csv
1,1/15/2023 12:05:00 AM,LUZON,01AEC_L01,OK,6553.8876,-37.7334,6647.619,-93.7314,0.0,NaN,DIPCER_202301150100.csv
2,1/15/2023 12:05:00 AM,LUZON,01AMBUK_SS,OK,6637.6476,0.0000,6647.619,-9.9714,0.0,NaN,DIPCER_202301150100.csv
3,1/15/2023 12:05:00 AM,LUZON,01AMBUK_T1L1,OK,6637.6476,0.0000,6647.619,-9.9714,0.0,NaN,DIPCER_202301150100.csv
4,1/15/2023 12:05:00 AM,LUZON,01AMBUK_U01,OK,6637.6476,0.0000,6647.619,-9.9714,0.0,NaN,DIPCER_202301150100.csv


In [4]:
#System GWAP only uses Visayas and Luzon
DIPC2 = DIPC[(DIPC['REGION_NAME']!= 'MINDANAO')&(DIPC['SCHED_MW']>=0)].copy()
DIPC2.tail()

,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,PRICING_FLAG,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,filename
1057330,1/19/2023,VISAYAS,08STBPB1_U02,OK,4932.9605,0.0,4771.2163,161.7442,0.0,NaN,DIPCER_202301190000.csv
1057331,1/19/2023,VISAYAS,08STBPB1_U03,OK,4932.9605,0.0,4771.2163,161.7442,0.0,NaN,DIPCER_202301190000.csv
1057332,1/19/2023,VISAYAS,08STBPB1_U04,OK,4932.9605,0.0,4771.2163,161.7442,0.0,NaN,DIPCER_202301190000.csv
1057333,1/19/2023,VISAYAS,08SUWECO_G01,OK,4932.9605,5.0,4771.2163,161.7442,0.0,NaN,DIPCER_202301190000.csv
1057334,1/19/2023,VISAYAS,08TIMBA_G01,OK,5110.4497,0.0,4771.2163,339.2335,0.0,NaN,DIPCER_202301190000.csv


In [5]:
#Solving for weighted price by multiplying price by scheduled load
DIPC2['SKEDXLMP'] = DIPC2['SCHED_MW']*DIPC2['LMP']


In [6]:
#summing all rows by time interval
DIPC4 = DIPC2.groupby(['TIME_INTERVAL'],sort=False).sum().copy()


In [7]:
#Solving for GWAP by dividing by total scheduled load per interval
DIPC4['GWAP'] = DIPC4['SKEDXLMP']/DIPC4['SCHED_MW']
DIPC4.head()

,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,SKEDXLMP,GWAP
TIME_INTERVAL,,,,,,,,
1/15/2023 12:05:00 AM,3.222531e+06,9063.0950,3.237390e+06,-14859.4222,0.0,0.0,5.789237e+07,6387.703794
1/15/2023 12:10:00 AM,2.995869e+06,9063.0049,3.010803e+06,-14934.0806,0.0,0.0,5.382109e+07,5938.547628
1/15/2023 12:15:00 AM,2.514585e+06,9018.2297,2.523368e+06,-8783.0820,0.0,0.0,4.492566e+07,4981.649376
1/15/2023 12:20:00 AM,2.866247e+06,9035.4807,2.880284e+06,-14036.5051,0.0,0.0,5.135714e+07,5683.941516
1/15/2023 12:25:00 AM,2.865086e+06,9032.7093,2.880284e+06,-15197.4882,0.0,0.0,5.132396e+07,5682.011271


In [8]:
#solving 3 day rolling average
DIPC4['AVERAGE_5_DAY'] = DIPC4['GWAP'].rolling(864).mean()
DIPC4.tail()

,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,SKEDXLMP,GWAP,AVERAGE_5_DAY
TIME_INTERVAL,,,,,,,,,
1/18/2023 11:40:00 PM,2.362834e+06,9138.8924,2.375105e+06,-12271.7004,0.0,0.0,4.257110e+07,4658.233785,7188.803270
1/18/2023 11:45:00 PM,2.350362e+06,9092.6634,2.358261e+06,-7899.2113,0.0,0.0,4.218330e+07,4639.267721,7187.578636
1/18/2023 11:50:00 PM,2.344494e+06,9043.2022,2.352667e+06,-8172.8773,0.0,0.0,4.186178e+07,4629.088597,7186.343139
1/18/2023 11:55:00 PM,2.355298e+06,9011.5234,2.356982e+06,-1684.4488,0.0,0.0,4.175685e+07,4633.716677,7185.114983
1/19/2023,2.339325e+06,8950.1898,2.342667e+06,-3341.7589,0.0,0.0,4.122521e+07,4606.071073,7184.316733


In [9]:
DIPC4.to_excel('DIPC.xlsx')
DIPC4.shape

(1152, 9)